In [6]:
from utilities import remove_cations
from utilities import remove_deliminators
from data_extract import get_molecule_cid
from data_extract import get_molecular_smiles
from utilities import get_l_max
from utilities import get_em_max
import os
import pandas as pd
import numpy as np

In [7]:
import pubchempy as pcp 
def get_molecule(file_name):
    my_file = file_name.split("_")
    cas = my_file[1]
    try:
        compound = pcp.get_compounds(cas, 'name')[0]
    except:
        try:
            my_name = cas.split(".") 
            compound = pcp.get_compounds(my_name[0], 'name')[0] 
        except:
            compound = None
    return compound

In [8]:
df = pd.read_csv('../data/PCAD3_Compd_Database_2018/2018_03_PCAD3.db', sep='\t',engine='python')

In [9]:
df.head()

,#,Absorption,Name,Structure,Class,File,Wavelength,Epsilon,Solvent,Instrument,...,Inv,Emission,File.1,Solvent.1,Quantum Yield,Instrument.1,Date.1,Reference.1,Inv.1,Unnamed: 21
0,1,NaN,Benzene,A01_71-43-2_Benzene.str.tif,Aromatic Hydrocarbons,A01_71-43-2_Benzene.abs.txt,254.75,210,cyclohexane,Cary 3,...,Ruchun A. Fuh,NaN,A01_71-43-2_Benzene.ems.txt,hexane,0.053,PTI QM-4/2003 SE,1/26/2009,"0.053 in hexane (Dawson, 1968), additional ref...",Masahiko Taniguchi,NaN
1,2,NaN,Toluene,A02_108-88-3_Toluene.str.tif,Aromatic Hydrocarbons,A02_108-88-3_Toluene.abs.txt,261.75,"2,860",cyclohexane,Cary 3,...,Ruchun A. Fuh,NaN,A02_108-88-3_Toluene.ems.txt,cyclohexane,0.170,Spex FluoroMax,"summer, 1995","0.17 in cyclohexane (Berlman, 1971), additiona...",Ruchun A. Fuh,NaN
2,3,NaN,o-Xylene,A03_95-47-6_o-Xylene.str.tif,Aromatic Hydrocarbons,A03_95-47-6_o-Xylene.abs.txt,263.00,254,cyclohexane,Agilent 8453,...,Masahiko Taniguchi,NaN,A03_95-47-6_o-Xylene.ems.txt,hexane,0.170,PTI QM-4/2003 SE,"January, 2017","0.17 in hexane (Froehlic, 1972), additional re...",Masahiko Taniguchi,NaN
3,4,NaN,m-Xylene,A04_108-38-3_m-Xylene.str.tif,Aromatic Hydrocarbons,A04_108-38-3_m-Xylene.abs.txt,265.00,284,cyclohexane,Agilent 8453,...,Masahiko Taniguchi,NaN,A04_108-38-3_m-Xylene.ems.txt,hexane,0.130,PTI QM-4/2003 SE,"January, 2017","0.13 in hexane (Froehlic, 1972), additional re...",Masahiko Taniguchi,NaN
4,5,NaN,p-Xylene,A05_106-42-3_p-Xylene.str.tif,Aromatic Hydrocarbons,A05_106-42-3_p-Xylene.abs.txt,275.00,770,cyclohexane,Agilent 8453,...,Masahiko Taniguchi,NaN,A05_106-42-3_p-Xylene.ems.txt,hexane,0.220,PTI QM-4/2003 SE,"January, 2017","0.22 in hexane (Froehlic, 1972), additional re...",Masahiko Taniguchi,NaN


In [10]:
df.columns

Index(['#', 'Absorption', 'Name', 'Structure', 'Class', 'File', 'Wavelength',
       'Epsilon', 'Solvent', 'Instrument', 'Date', 'Reference', 'Inv',
       'Emission', 'File.1', 'Solvent.1', 'Quantum Yield', 'Instrument.1',
       'Date.1', 'Reference.1', 'Inv.1', 'Unnamed: 21'],
      dtype='object')

### clean epsilon

In [11]:
df['Epsilon']=df['Epsilon'].str.replace(',','').astype(int)  # clean epsilon

In [12]:
clean_df = pd.DataFrame()
columns_kept = ['#','Name', 'Epsilon', 'Solvent', 'Quantum Yield','Solvent.1', 'File','File.1']
clean_df[columns_kept] = df[columns_kept]
clean_df['Absorption'] = df['Wavelength']


In [13]:
clean_df['SMILES'] = 'NaN'
for i in range(df.shape[0]):
    compound = get_molecule(df['File'].loc[i])
    if compound is not None:
        clean_df['SMILES'].loc[i] = compound.isomeric_smiles

C:\Users\Yang\.conda\envs\test_speedcom\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [14]:
for i in range(clean_df.shape[0]):
    clean_df['SMILES'].loc[i] = remove_cations(clean_df['SMILES'].loc[i])

In [15]:
clean_df_drop_no_smiles = clean_df[clean_df['SMILES'] != 'NaN']

In [16]:
clean_df_drop_no_smiles

,#,Name,Epsilon,Solvent,Quantum Yield,Solvent.1,File,File.1,Absorption,SMILES
0,1,Benzene,210,cyclohexane,0.0530,hexane,A01_71-43-2_Benzene.abs.txt,A01_71-43-2_Benzene.ems.txt,254.75,C1=CC=CC=C1
1,2,Toluene,2860,cyclohexane,0.1700,cyclohexane,A02_108-88-3_Toluene.abs.txt,A02_108-88-3_Toluene.ems.txt,261.75,CC1=CC=CC=C1
2,3,o-Xylene,254,cyclohexane,0.1700,hexane,A03_95-47-6_o-Xylene.abs.txt,A03_95-47-6_o-Xylene.ems.txt,263.00,CC1=CC=CC=C1C
3,4,m-Xylene,284,cyclohexane,0.1300,hexane,A04_108-38-3_m-Xylene.abs.txt,A04_108-38-3_m-Xylene.ems.txt,265.00,CC1=CC(=CC=C1)C
4,5,p-Xylene,770,cyclohexane,0.2200,hexane,A05_106-42-3_p-Xylene.abs.txt,A05_106-42-3_p-Xylene.ems.txt,275.00,CC1=CC=C(C=C1)C
5,6,Mesitylene,190,cyclohexane,0.0880,hexane,A06_108-67-8_Mesitylene.abs.txt,A06_108-67-8_Mesitylene.ems.txt,265.00,CC1=CC(=CC(=C1)C)C
6,7,Durene,214,cyclohexane,0.3000,hexane,A07_95-93-2_Durene.abs.txt,A07_95-93-2_Durene.ems.txt,278.00,CC1=CC(=C(C=C1C)C)C
7,8,Pentamethylbenzene,629,cyclohexane,0.0750,hexane,A08_700-12-9_Pentamethylbenzene.abs.txt,A08_700-12-9_Pentamethylbenzene.ems.txt,271.00,CC1=CC(=C(C(=C1C)C)C)C
8,9,Hexamethylbenzene,214,cyclohexane,0.0100,hexane,A09_ 87-85-4_Hexamethylbenzene.abs.txt,A09_ 87-85-4_Hexamethylbenzene.ems.txt,271.00,CC1=C(C(=C(C(=C1C)C)C)C)C
9,10,Phenol,2340,cyclohexane,0.0750,hexane,A10_108-95-2_Phenol.abs.txt,A10_108-95-2_Phenol.ems.txt,270.75,C1=CC=C(C=C1)O


In [17]:
import os

In [18]:
em_max = get_em_max(clean_df_drop_no_smiles, 'File.1', prefix_dir)

In [20]:
clean_df_drop_no_smiles['Emission']=em_max

C:\Users\Yang\.conda\envs\test_speedcom\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [21]:
clean_df_drop_no_smiles

,#,Name,Epsilon,Solvent,Quantum Yield,Solvent.1,File,File.1,Absorption,SMILES,Emission
0,1,Benzene,210,cyclohexane,0.0530,hexane,A01_71-43-2_Benzene.abs.txt,A01_71-43-2_Benzene.ems.txt,254.75,C1=CC=CC=C1,287.0
1,2,Toluene,2860,cyclohexane,0.1700,cyclohexane,A02_108-88-3_Toluene.abs.txt,A02_108-88-3_Toluene.ems.txt,261.75,CC1=CC=CC=C1,289.5
2,3,o-Xylene,254,cyclohexane,0.1700,hexane,A03_95-47-6_o-Xylene.abs.txt,A03_95-47-6_o-Xylene.ems.txt,263.00,CC1=CC=CC=C1C,291.0
3,4,m-Xylene,284,cyclohexane,0.1300,hexane,A04_108-38-3_m-Xylene.abs.txt,A04_108-38-3_m-Xylene.ems.txt,265.00,CC1=CC(=CC=C1)C,290.0
4,5,p-Xylene,770,cyclohexane,0.2200,hexane,A05_106-42-3_p-Xylene.abs.txt,A05_106-42-3_p-Xylene.ems.txt,275.00,CC1=CC=C(C=C1)C,291.0
5,6,Mesitylene,190,cyclohexane,0.0880,hexane,A06_108-67-8_Mesitylene.abs.txt,A06_108-67-8_Mesitylene.ems.txt,265.00,CC1=CC(=CC(=C1)C)C,290.0
6,7,Durene,214,cyclohexane,0.3000,hexane,A07_95-93-2_Durene.abs.txt,A07_95-93-2_Durene.ems.txt,278.00,CC1=CC(=C(C=C1C)C)C,295.0
7,8,Pentamethylbenzene,629,cyclohexane,0.0750,hexane,A08_700-12-9_Pentamethylbenzene.abs.txt,A08_700-12-9_Pentamethylbenzene.ems.txt,271.00,CC1=CC(=C(C(=C1C)C)C)C,295.0
8,9,Hexamethylbenzene,214,cyclohexane,0.0100,hexane,A09_ 87-85-4_Hexamethylbenzene.abs.txt,A09_ 87-85-4_Hexamethylbenzene.ems.txt,271.00,CC1=C(C(=C(C(=C1C)C)C)C)C,298.0
9,10,Phenol,2340,cyclohexane,0.0750,hexane,A10_108-95-2_Phenol.abs.txt,A10_108-95-2_Phenol.ems.txt,270.75,C1=CC=C(C=C1)O,291.5


In [22]:
clean_df_drop_no_smiles.to_csv('../data/PCAD3_Compd_Database_2018/cleaned_data.tsv',sep='\t', index=False)

### below functions moved to utils

In [23]:
# moved to utilities
def get_l_max(wavelength_intensity):
    """
    args:
    wavelength_intensity -- np.array(2D), dtypes float64
        first column: wavelength; seconde column: intensity
    return:
    np.array([lambda_max(float), intensity])
    """
    wavelength_intensity.view('f8,f8').sort(order=['f1'], axis = 0)
    return wavelength_intensity[-1][0]

In [24]:
#moved to utilities
def get_em_max(clean_df, em_file_colname,prefix_dir):
    """
    from list of emission file names, get the lambda max from existing files 
    and fill None if file not exist
    """
    from data_extract import get_spectra, get_peaks
    emission=[]
    for x in clean_df[em_file_colname].astype(str):
        if x != 'nan':
            em_max = get_l_max(get_peaks(get_spectra(os.path.join(prefix_dir, x))))
            emission.append(em_max)
        else:
            emission.append(None)
    return emission